In [12]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2648).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./cl_classes1307.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../best_model2306/pytorch_model10d.bin", map_location=device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [13]:
#импорт описания классов (Алексей слямзил)
dscr = pd.read_csv("../data/descr10d.csv", sep=';', names=['id', 'label', 'till'], dtype={'id': str, 'label': str, 'till': str})

In [101]:
#dscr['till'] = pd.to_datetime(dscr['till'], format='%Y-%m-%d')
#dscr['label'] = dscr['label'].str.lower()
#dscr['label'] = dscr['label'].str.strip()
#dscr.to_csv('../data/descr10d.csv', sep=';', columns=['id', 'label', 'till'], index=False, header=False)

In [14]:
dscr['valid'] = dscr['till'].isna()

In [45]:
#Устаревшие коды
len(dscr[dscr['till'].notna()])

240

In [46]:
#точки данных с устаревшими кодами
len(df[df['id'].isin(dscr[dscr['till'].notna()]['id'])])

3529

In [15]:
#функция предсказания одного класса (аргмакс). По умолчанию с кодом класса дает описание класса
def predict_class(text, desc=True):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if desc==True:
            result = dict()
            id = Label_encoder.inverse_transform([predicted_class_id])[0]
            result[id] = dscr[dscr['id']==id].iloc[0]['label']
            return result
        else:
            return Label_encoder.inverse_transform([predicted_class_id])[0]

In [24]:
#Пример с описанием
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО')

{'1704906500': 'Кондитерские изделия в виде резинки и желе, включая фруктовую пасту в виде кондидерских изделий из сахара, не содержащие какао'}

In [19]:
#Пример без описания
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО', desc=False)

'1704906500'

In [16]:
#функция предикта классов с вероятностью (софтмакс), по умолчанию 5 классов
def predict_prob(text, qtty=5):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [71]:
#что такое логиты:
inputs = tokenizer("Пластмассовый мир победил", truncation = True, max_length=100, padding='max_length', return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
print("logits: ", logits)
print("logits shape: ", logits.shape)
print("logits mean value: ", torch.mean(logits))
print("logits standart deviation: ", torch.std(logits))
print("logits summ: ", torch.sum(logits))
sm = torch.nn.functional.softmax(logits, dim=1)
print("softmax: ", sm)
print("softmax shape: ", sm.shape)
print("softmax mean value: ", torch.mean(sm))
print("softmax standart deviation: ", torch.std(sm))
print("softmax summ: ", torch.sum(sm))

logits:  tensor([[-3.2495, -1.9287, -0.6356,  ...,  1.2311,  1.4205,  1.4541]])
logits shape:  torch.Size([1, 2648])
logits mean value:  tensor(-0.4165)
logits standart deviation:  tensor(1.4571)
logits summ:  tensor(-1102.8923)
softmax:  tensor([[3.6373e-06, 1.3628e-05, 4.9658e-05,  ..., 3.2113e-04, 3.8812e-04,
         4.0139e-04]])
softmax shape:  torch.Size([1, 2648])
softmax mean value:  tensor(0.0004)
softmax standart deviation:  tensor(0.0042)
softmax summ:  tensor(1.)


In [59]:
torch.sum(logits)

tensor(-664.6194)

In [17]:
#функция предикта классов и описания с вероятностью, по умолчанию 5 шт. Использует функцию predict_prob.
#возвращает список списков
def predict_prob_with_descr(text, qtty=5):
    probs = predict_prob(text, qtty=qtty)
    #result = np.array()
    result = list()
    for each in probs:
        #valid = np.nonzero(dscr[dscr['id']==each].iloc[0]['till'])
        
        result.append([each, dscr[dscr['id']==each].iloc[0]['label'], probs[each], dscr[dscr['id']==each].iloc[0]['valid']])
    return result


In [11]:
#Пример с 3 классами
print(predict_prob_with_descr('шины пневматические резиновые новые', 3))

[['4011800000', 'шины и покрышки пневматические резиновые новые для транспортных средств и машин, используемых в строительстве, горном деле или промышленности', 0.9863452315330505, True], ['4011209000', 'шины и покрышки пневматические резиновые новые, для автобусов или моторных транспортных средств для перевозки грузов с индексом нагрузки более 121', 0.004065710585564375, True], ['4011900000', 'прочие шины и покрышки пневматические резиновые новые', 0.0034278042148798704, True]]


In [85]:
#Пример с 5 классами
print(predict_prob_with_descr('семена тыквы ', 5))

[['1207999600', 'семена и плоды прочих масличных культур, дробленые или недробленые, прочие', 0.9361142516136169, False], ['2008199900', 'прочие орехи, арахис и прочие семена, смешанные или не смешанные между собой в первичных упаковках нетто-массой не более 1 кг', 0.010454857721924782, True], ['1207409000', 'прочие семена кунжута, дробленые или недробленые', 0.00979942362755537, True], ['0711400000', 'огурцы и корнишоны, консервированные для кратковременного хранения (например, диоксидом серы, в рассоле, сернистой воде или в другом временно консервирующем растворе), но в таком виде непригодные для непосредственного употребления в пищу', 0.0006681593949906528, True], ['2001902000', 'плоды рода capsicum, кроме перца стручкового сладкого или перца душистого, приготовленные или консервированные с добавлением уксуса или уксусной кислоты', 0.0006370716728270054, True]]


In [28]:
#причесанный в пандас пример
pd.DataFrame(predict_prob_with_descr('пластмассовые болты', 15), columns=['ID', 'LABEL', 'PROB', 'TILL'])

,ID,LABEL,PROB,TILL
0,7318156900,"винты и болты прочие, в комплекте с гайками ил...",0.209226,True
1,7318158100,"винты и болты прочие, в комплекте с гайками ил...",0.195239,True
2,7318159009,"прочие винты и болты, с головками, из черных м...",0.061655,False
3,7318190008,"изделия, снабженные резьбой, прочие",0.032003,True
4,7318159008,"винты и болты прочие, в комплекте с гайками ил...",0.028338,True
5,7318149900,"винты прочие самонарезающие, снабженные резьбо...",0.024115,True
6,7616100000,"гвозди, кнопки, скобы (кроме указанных в товар...",0.018865,True
7,8311300000,прутки с покрытием и проволока с сердечникомб ...,0.016887,True
8,7318155900,"винты и болты прочие, в комплекте с гайками ил...",0.015579,True
9,7318154100,"винты и болты прочие, в комплекте с гайками ил...",0.013955,True


In [83]:
def consolelog(probs):
    print('prob-ty', 'Code', 'Description', sep=':    ')
    #print('____________________________________________________________________')
    for each in probs:
        if len(each[1]) < 40:
            des = each[1]
        else:
            des = each[1][:40] + '...'
        print(f"{each[2]:.{5}f}", each[0], des, sep=':    ')
        #print('____________________________________________________________________')

In [84]:
consolelog(predict_prob_with_descr("аттракционы", 5))

prob-ty:    Code:    Description
0.00686:    8714993000:    багажники к транспортным средствам товар...
0.00610:    8541401000:    светоизлучающие диоды (led), включая лаз...
0.00571:    9508900000:    прочие карусели, качели, тиры и прочие а...
0.00551:    8529906509:    электронные модули, предназначенные искл...
0.00536:    9508260000:    аттракционы водных парков


In [18]:
#загрузим обучающий датасет
df = pd.read_csv("../data/mergedcleared1307.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#df = pd.read_csv("../data/firstset.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

In [30]:
#пример использования по полному коду (можно делать по части кода)
whats_data('7318156900', pandas=False)

array([['7318156900',
        'inner hexagonal floriated countersunk head screw винт с потайной головкой и внутренним шестигранником 240шт'],
       ['7318156900',
        'jq2580822t1f32 inner hexagonal floriated countersunk head screw винт с потайной головкой и внутренним шестигранником 960ед'],
       ['7318156900',
        'винты прочие из углеродистой стали с внутренним шестигранником'],
       ['7318156900',
        'запасные части и принадлежности для ремонта и обслуживания велосипедов из черных металлов комплект крепежных болтов для тормоза на раму с шестигранным шлицем 23упак'],
       ['7318156900',
        'запасные части и принадлежности для ремонта и обслуживания велосипедов из черных металлов сменные болты для велосипедных шатунов 1упаковка'],
       ['7318156900',
        'запасные части и принадлежности для ремонта и обслуживания велосипедов ссменные болты для велосипедного выноса 10 комплектов сменный болт для системы шатунов 6 пар'],
       ['7318156900',
        'изд

In [22]:
#еще пример
df[df['id'].isin(list(whats_data('3926909709')['id'].unique()))]

,id,label
134518,3926909709,1018005048 plug пластмассовая заглушка 240ед
134519,3926909709,1018059237 эмблема geely product external mark...
134520,3926909709,6600005537 водонепроницаемая пленка левой пере...
134521,3926909709,6600007488 левая задняя виброизоляционная пане...
134522,3926909709,6600007504 левая центральная демпфирующая плас...
...,...,...
165501,3926909709,скребок из пластмассы для очистки автомобиля о...
165502,3926909709,трубчатый зажим двойной double pipe clamp
165503,3926909709,форсунка стеклоомывателя задняя nl 3b rear was...
165504,3926909709,части пластмассовые для швейной машины всего 3...


In [41]:

plast[plast['label'].str.contains("винт")==True]

,id,label
147487,3926909709,изделия прочие из пластмасс клипса для креплен...


In [44]:
plast = whats_data('3926909709', pandas=True)
plast[plast['label'].str.contains("болт")==True].to_numpy()

array([['3926909709',
        'protector door hinge screw колпачок болта дверного шарнира'],
       ['3926909709',
        'protector door hinge screw колпачок болта дверного шарнира single pipe clamp зажим одной трубки plug пластмассовая заглушка'],
       ['3926909709',
        'изделия прочие из пластмасс и изделия из прочих материалов товарных позиций 3901 3914 пластмассовый ящик из полипропилена предназначен для хранения болтов гаек и прочей мелочи всего 100 ед'],
       ['3926909709',
        'пылезащитная крышка из пластмассы выходного вала редуктора предн для червячного редуктора серии rv в комплекте с болтами'],
       ['3926909709',
        'protector door hinge screw колпачок болта дверного шарнира 960шт'],
       ['3926909709',
        'protector door hinge screw колпачок болта дверного шарнира960 шт'],
       ['3926909709',
        'инструмент для демонтажа декоративной крышки колесного болта removal tool wheel bolt screw cap 120 шт'],
       ['3926909709',
        '106802

In [72]:
similar = pd.read_csv("../data/Similar.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
similar

,id,label
0,4011100009,шины
1,4011100009,шина
2,4011100009,авто шины
3,4011100009,чернитель шин
4,4011100009,тапочки резиновые
...,...,...
59752,8474100000,грохот sandvik
59753,8474100000,вибрационный стол
59754,8474100000,вибрационные грохоты для горнодобывающей промы...
59755,8474100000,аппарат для деления руды


In [79]:
similar['labels'] = Label_encoder.transform(similar['id'])

In [76]:
#result[Label_encoder.inverse_transform([a])[0]]
Label_encoder.transform(['8474100000'])

array([1748])

In [81]:
Label_encoder.inverse_transform([458])

array(['4011100009'], dtype=object)

In [86]:
similar.columns = ['id', 'description', 'labels']
similar = similar.drop(['id'], axis=1, inplace=False)
similar

,description,labels
0,шины,458
1,шина,458
2,авто шины,458
3,чернитель шин,458
4,тапочки резиновые,458
...,...,...
59752,грохот sandvik,1748
59753,вибрационный стол,1748
59754,вибрационные грохоты для горнодобывающей промы...,1748
59755,аппарат для деления руды,1748


In [87]:
similar.to_csv('../../../test.csv', index=False)

In [88]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'test': '../../../test.csv'})

Extracting data files: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 249.75it/s]


Dataset csv downloaded and prepared to C:/Users/yeril/.cache/huggingface/datasets/csv/default-18c41634c0c9b476/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 124.55it/s]


In [89]:
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)

In [91]:
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
#dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))

In [92]:
set(dataset['test'].features.keys())


{'attention_mask', 'description', 'input_ids', 'labels', 'token_type_ids'}

In [130]:
dataset.shape

{'test': (59757, 5)}

In [129]:
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 

In [131]:
from torch.utils.data import DataLoader
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=2)


In [97]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [98]:
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

In [100]:
from tqdm import tqdm

In [132]:
from sklearn import preprocessing
from sklearn.metrics import classification_report
#Label_encoder = preprocessing.LabelEncoder()
#Label_encoder.classes_ = np.load('./cl_classes1307.npy', allow_pickle=True)

true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

100%|████████████████████████████████████████████████████████████████████████████| 29879/29879 [53:32<00:00,  9.30it/s]
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Pyt

              precision    recall  f1-score   support

  0207149909       0.87      0.26      0.40        50
  0304999800       0.07      1.00      0.13         2
  0305542000       1.00      1.00      1.00         6
  0305549000       0.00      0.00      0.00         0
  0305599000       0.94      0.66      0.77        44
  0604909900       0.13      0.55      0.21        11
  0701905000       0.30      1.00      0.46         3
  0703200000       0.80      0.50      0.62        16
  0711400000       0.00      0.00      0.00         0
  0711590000       0.00      0.00      0.00         0
  0712909000       0.59      0.74      0.66        31
  0713339000       0.24      1.00      0.39         7
  0801110000       0.45      0.42      0.43        12
  0802310000       0.64      1.00      0.78         7
  0802320000       1.00      0.67      0.80        12
  0803901000       0.70      0.50      0.58        14
  0804300009       0.00      0.00      0.00         0
  0804500001       0.88    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [173]:
x = Label_encoder.inverse_transform(true)

In [177]:
y = Label_encoder.inverse_transform(preds)

In [178]:
real = pd.DataFrame(x, columns=['id'])
real['label'] = real.id.str.slice(start=0, stop=4)
predicted = pd.DataFrame(y, columns=['id'])
predicted['label'] = predicted.id.str.slice(start=0, stop=4)

In [180]:
print(classification_report(real['label'], predicted['label']))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: Undefi

              precision    recall  f1-score   support

        0207       0.87      0.26      0.40        50
        0304       0.07      1.00      0.13         2
        0305       0.90      0.72      0.80        50
        0604       0.13      0.55      0.21        11
        0701       0.30      1.00      0.46         3
        0703       0.80      0.50      0.62        16
        0711       0.00      0.00      0.00         0
        0712       0.59      0.74      0.66        31
        0713       0.24      1.00      0.39         7
        0801       0.45      0.42      0.43        12
        0802       1.00      1.00      1.00        19
        0803       0.70      0.50      0.58        14
        0804       0.67      0.68      0.68        38
        0805       0.91      0.49      0.64        98
        0806       0.95      0.73      0.83        26
        0808       0.96      0.57      0.72       119
        0813       0.00      0.00      0.00         0
        0901       0.96    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#10 digits
               precision    recall  f1-score   support


    accuracy                           0.24     59757
   macro avg       0.27      0.21      0.19     59757
weighted avg       0.37      0.24      0.25     59757

In [ ]:
#4 digits
               precision    recall  f1-score   support

    accuracy                           0.45     59757
   macro avg       0.42      0.38      0.35     59757
weighted avg       0.53      0.45      0.46     59757

In [187]:
similar['description']

0                                                     шины
1                                                     шина
2                                                авто шины
3                                            чернитель шин
4                                        тапочки резиновые
                               ...                        
59752                                       грохот sandvik
59753                                    вибрационный стол
59754    вибрационные грохоты для горнодобывающей промы...
59755                             аппарат для деления руды
59756                                       вибродвигатель
Name: description, Length: 59757, dtype: object